# Brainstat analysis comparing 3T and 7T epilepsy  

Surface-based comparisons  
- vertex-wise T-test : are controls and pts different
    - Use brainstat
- vertex-wise effect size : how big are the distances between the vertex differences?
    - Use own function


For figures: 
- Visualize effect size on a brain masked for significant p-values 

In [54]:
import pandas as pd
import numpy as np
import nibabel as nib
import pickle
import datetime
import brainstat as bstat

In [ ]:
# specify root directories
MICs = {
    "name": "MICs",
    "dir_root": "/data/mica3/BIDS_MICs",
    "dir_mp": "micapipe_v0.2.0",
    "dir_hu": "hippunfold_v1.3.0/hippunfold",
    "study": "3T",
    "ID_ctrl" : ["HC"],
    "ID_Pt" : ["PX"]
    }

PNI = {
    "name": "PNI",
    "dir_root": "/data/mica3/BIDS_PNI",
    "dir_mp": "micapipe_v0.2.0",
    "dir_hu": "hippunfold_v1.3.0/hippunfold",
    "study": "7T",
    "ID_col" : ["PNC", "Pilot"], # column for ID in demographics file
    }

studies = [MICs, PNI]

demographics = {
    "pth" : "/host/verges/tank/data/daniel/3T7T/z/data/pt/demo_22May2025.csv",
    # column names:
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "grp" : "grp_detailed" # col name for participant grouping variable of interest
}

px_grps = { # specify patient group labels to compare to controls
    'all' : ['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'],
    'TLE' : ['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'],
    'TLE_L': ['TLE_L', 'mTLE_L'],
    'TLE_R': ['TLE_R', 'mTLE_R'],
    'FCD' : ['FLE_R', 'FLE_L'],
    'MFCL' : ['MFCL', 'MFCL_bTLE'],
    'UKN' : ['UKN_L', 'UKN_U']
}

ctrl_grp = {'ctrl' : ['CTRL']}

groups = {**px_grps, **ctrl_grp} # Combine all patient and control groups into a single dictionary

surfaces = ["fsLR-5k"]
labels = ["pial", "white", "midThick"]

{'all': ['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'], 'TLE': ['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'], 'TLE_L': ['TLE_L', 'mTLE_L'], 'TLE_R': ['TLE_R', 'mTLE_R'], 'FCD': ['FLE_R', 'FLE_L'], 'MFCL': ['MFCL', 'MFCL_bTLE'], 'UKN': ['UKN_L', 'UKN_U'], 'ctrl': ['CTRL']}


In [5]:
# retrieve surfaces from pt of interest
def load_surf(study, IDss):
    """
    Get the surface data for a given group.

    inputs:
    study: dictionary item with keys 'name', 'dir_root', 'study'
    IDs: pd.dataframe woth cols IDs and SES indicating all participants IDs to extract surfaces for

    outputs:
    surfs: pd.dataframe with vertices in rows and unique ID_SES in columns
    """
    
    import nibabel as nib
    
    # get the list of patients in the group
    pt_list = bstat.get_subjects(study, pt_grp)
    
    # get the surface data for each patient
    surf_data = {}
    for pt in pt_list:
        surf_data[pt] = {}
        for surf in surfaces:
            surf_data[pt][surf] = bstat.get_surface_data(study, pt, surf)
    
    return surf_data

In [66]:
def chk_pth(pth):
    """
    Check if the path exists and is a file.
    
    inputs:
        pth: path to check
    
    outputs:
        True if the path exists and is a file, False otherwise
    """
    
    import os
    
    if os.path.exists(pth) and os.path.isfile(pth):
        return True
    else:
        return False

In [75]:
# when working, add to Utils scripts
def get_1surfPath(root, deriv_fldr, sub, ses, label="midthickness", surf="fsLR-5k", space="nativepro", hemi="LR", check_pth=True,silence=True):
    """
    Get the path to the surface data for a given subject and session.
    Assumes BIDS format of data storage.

    inputs:
        root: root directory of the study
        deriv_fldr: name of derivative folder containing the surface data
        sub: subject ID (no `sub-` prefix)
        ses: session ID (with leading zero if applicable; no `ses-` prefix)
        surf: surface type and resolution (e.g., fsLR-32k, fsLR-5k)
        label: surface label (e.g., "pial", "white", "midThick")
        space: space of the surface data (e.g., "nativepro", "fsnative")
        hemi: hemisphere to extract (default is "LR" for both left and right hemispheres)

        check_pth: whether to check if the path exists (default is True)
        silence: whether to suppress print statements (default is True)
    outputs:
        path to the surface data files
    """

    # make surf to lower case
    label = label.lower()

    # ensure that surface is well defined
    if label == "pial":
        label = "pial"
    elif label == "white":
        label = "white"
    elif label == "midThick" or label == "midthickness":
        label = "midthickness"
    else:
        raise ValueError("Invalid surface type. Choose from 'pial', 'white', or 'midThick'.")
    
    # construct the path to the surface data file
    
    pth = f"{root}/derivatives/{deriv_fldr}/sub-{sub}/ses-{ses}/surf"
    
    hemi = hemi.upper()
    if hemi == "LEFT" or hemi == "L":
        hemi = "L"
    elif hemi == "RIGHT" or hemi == "R":
        hemi = "R"
    elif hemi != "LR":
        raise ValueError("Invalid hemisphere. Choose from 'L', 'R', or 'LR'.")

    # handle hippunfold naming convention
    if "hippunfold" in deriv_fldr.lower():
        # space usually: "T1w"
        # surf usually: "2mm"
        # label options: "hipp_outer", "hipp_inner", "hipp_midthickness"

        if hemi == "LR":
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_den-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_den-{surf}_label-{label}.surf.gii"
            pth = [pth_L, pth_R]
            
            if not silence: print(f"[surf_pth] Returning hippunfold paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_den-{surf}_label-{label}.surf.gii"

    elif "micapipe" in deriv_fldr.lower():
        if hemi == "LR":
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_surf-{surf}_label-{label}.surf.gii"
        
            pth = [pth_L, pth_R]
            if not silence: print(f"[surf_pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth = [pth_L, pth_R]
            if not silence: print(f"[surf_pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_surf-{surf}_label-{label}.surf.gii"

    if check_pth:
        if isinstance(pth, list):
            for idx, p in enumerate(pth):
                if not chk_pth(p):
                    print(f"\t[get_1surfPth] FILE NOT FOUND (sub-{sub}_ses-{ses}): {p}")
                    pth[idx] = "ERROR:" + p
        else:
            if not chk_pth(pth):
                print(f"\t[get_1surfPth] FILE NOT FOUND (sub-{sub}_ses-{ses}): {pth}")
                pth = "ERROR:" + pth
    
    return pth


def get_NsurfPaths(demographics, study, groups, label="midthickness", hemi="LR", space="nativepro", surf="fsLR-5k"):
    """
    Get path to surface files for individual groups


    Input:
    demographics: dict  regarding demographics file. 
        Required keys: 
            'pth'
            'ID_7T'
            'ID_3T'
            'SES'
            'grp'
    study: dict  regarding study.
        Required keys: 
            'name'
            'dir_root'
            'study'
            'dir_mp'
            'dir_hu'
    groups: dict    of groups to extract surfaces for. 
        Each key should be a group name, and the value should be a list of labels in the 'grp' column of demographics file assigned to that group.
    label: str  surface label to extract
    hemi: str  hemisphere to extract. Default is "LR" for both left and right hemispheres.
    space: str  space of the surface data. Default is "nativepro".
    surf: str  surface type and resolution. Default is "fsLR-5k".
    """
    import pandas as pd

    demo = pd.read_csv(demographics['pth'], dtype=str)
    
    out = []

    for grp_name, grp_labels in groups.items():
        print(f"{study['name']} {grp_name} ({grp_labels})")

        # get IDs for this group
        ids = demo.loc[
            (demo[demographics['grp']].isin(grp_labels)) &
            (demo['study'] == study['study']),
            [ID_col, demographics['SES'], 'study']
        ].copy()

        for i, row in ids.iterrows():
            ID = row[ID_col]
            SES = row[demographics['SES']]
            #print(f"\tsub-{ID}_ses-{SES}")
            pth = get_1surfPath(root=study['dir_root'], deriv_fldr=study['dir_mp'], sub=ID, ses=SES, label=label, surf=surf, space=space, hemi=hemi)
            # add this pth to the dataframe
            if isinstance(pth, list):
                ids.loc[i, f'surf_L'] = pth[0]
                ids.loc[i, f'surf_R'] = pth[1]
            else:
                ids.loc[i, f'surf_{hemi}'] = pth 
        
        # create dictionary item for each group, add to output list
        out.append({
            'study': study['name'],
            'grp': grp_name,
            'grp_labels': grp_labels,
            'surf_pths': ids
        })

    return out

In [27]:
demo = pd.read_csv(demographics['pth'], dtype=str)
demo[["MICS_ID", "PNI_ID", "study", "SES", "grp", "grp_detailed"]]

,MICS_ID,PNI_ID,study,SES,grp,grp_detailed
0,HC129,Pilot013,7T,05,CTRL,CTRL
1,HC082,PNC003,7T,01,CTRL,CTRL
2,HC082,PNC003,7T,02,CTRL,CTRL
3,HC082,PNC003,7T,03,CTRL,CTRL
4,HC082,PNC003,7T,04,CTRL,CTRL
...,...,...,...,...,...,...
116,HC130,PNC026,3T,02,CTRL,CTRL
117,HC083,PNC011,3T,02,CTRL,CTRL
118,PX215,PNE020,3T,01,UKN,UKN_U
119,PX216,PNE021,3T,01,TLE,TLE_R


In [76]:
surf_pths = []
if 'surf_dfs' not in locals():
    surf_dfs = {}

for study in studies:
    if study['study'] == "3T":
        ID_col = demographics['ID_3T']
    elif study['study'] == "7T":
        ID_col = demographics['ID_7T']

    surf_pths.extend(get_NsurfPaths(demographics, study, groups))

# save
save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/paths"
date = datetime.datetime.now().strftime("%d%b%Y")
with open(f'{save_pth}/surface_paths_{date}.pkl', 'wb') as f:
    pickle.dump(surf_pths, f)


MICs all (['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'])
	[get_1surfPth] FILE NOT FOUND (sub-PX216_ses-01): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX216/ses-01/surf/sub-PX216_ses-01_hemi-L_space-nativepro_surf-fsLR-5k_label-midthickness.surf.gii
	[get_1surfPth] FILE NOT FOUND (sub-PX216_ses-01): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX216/ses-01/surf/sub-PX216_ses-01_hemi-R_space-nativepro_surf-fsLR-5k_label-midthickness.surf.gii
MICs TLE (['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'])
	[get_1surfPth] FILE NOT FOUND (sub-PX216_ses-01): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX216/ses-01/surf/sub-PX216_ses-01_hemi-L_space-nativepro_surf-fsLR-5k_label-midthickness.surf.gii
	[get_1surfPth] FILE NOT FOUND (sub-PX216_ses-01): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX216/ses-01/surf/sub-PX216_ses-01_hemi-R_space-nativepro_surf-fsLR-5k_label-midthickness.surf.gii
MICs TLE_

In [ ]:
# load list of dict items with surface paths
#with open('surf_pths.pkl', 'rb') as f:
#    surf_pths = pickle.load(f)

In [ ]:
# print specific study-group combinations
get_study = "PNI"
get_grp = "TLE"

for item in surf_pths:
    if item['study'] == get_study and item['grp'] == get_grp:
        print(f"{item['study']}-{item['grp']} ({item['grp_labels']})")
        with pd.option_context('display.max_columns', None):
            print(item['surf_pths'])
        break

PNI-TLE (['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'])
    PNI_ID SES study                                             surf_L  \
45  PNE001  01    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
46  PNE001  01    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
47  PNE001  01    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
48  PNE001  01    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
53  PNE006  a1    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
59  PNE010  a1    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
60  PNE011  a1    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
61  PNE012  a1    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
62  PNE013  a1    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
66  PNE017  a1    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
67  PNE018  a1    7T  /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2...   
68  PNE019  a1    7T  /data/mica3/BIDS_PNI

In [60]:
# print paths
for item in surf_pths:
    if item['study'] == get_study and item['grp'] == get_grp:
        # print values of surf_L and/or surf_R columns if they exist
        if 'surf_L' in item['surf_pths'].columns:
            print(f"{item['surf_pths']['surf_L'].values[0]}")
        if 'surf_R' in item['surf_pths'].columns:
            print(f"{item['surf_pths']['surf_R'].values[0]}")

/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE001/ses-01/surf/sub-PNE001_ses-01_hemi-L_space-nativepro_surf-fsLR-5k_label-midthickness.surf.gii
/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE001/ses-01/surf/sub-PNE001_ses-01_hemi-R_space-nativepro_surf-fsLR-5k_label-midthickness.surf.gii


In [ ]:
# print structure of surf_pthshttps://file+.vscode-resource.vscode-cdn.net/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE001/ses-01/surf/sub-PNE001_ses-01_hemi-L_space-nativepro_surf-fsLR-5k_label-midthickness.surf.gii
for item in surf_pths:
    print(f"{item['study']}-{item['grp']}")
    print(item['surf_pths'])
    print()


In [ ]:
surf_dfs

In [ ]:
# flip TLEs --> put all lesions on same side

In [ ]:
# get difference maps at 3T (3T ctrl - 3T cases)

In [ ]:
# get difference maps at 7T (7T ctrl - 7T cases)

In [ ]:
# get difference maps of difference maps (3T dif maps - 7T dif maps)